In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, color
from skimage.measure import regionprops, regionprops_table
import pandas as pd
from scipy.ndimage import label
import cv2
import re

# Path definitions
input_folder = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Cell"
output_folder = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3_1.4Pa_18h"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def extract_pressure_from_filename(filename):
    """Extract pressure value from filename"""
    if "0Pa" in filename:
        return 0.0
    elif "1.4Pa" in filename:
        return 1.4
    else:
        return None

def get_cell_masks(folder_path):
    """Get all cell mask images and organize by pressure"""
    files_0pa = []
    files_1_4pa = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".tif") and "cell_mask" in filename:
            full_path = os.path.join(folder_path, filename)
            pressure = extract_pressure_from_filename(filename)

            if pressure == 0.0:
                files_0pa.append(full_path)
            elif pressure == 1.4:
                files_1_4pa.append(full_path)

    return files_0pa, files_1_4pa

def load_and_get_regions(image_path):
    """Load an image and get the labeled regions"""
    img = io.imread(image_path)

    # If image is RGB, convert to grayscale
    if len(img.shape) > 2 and img.shape[2] == 3:
        img = color.rgb2gray(img)

    # Ensure image is binary
    thresh = np.unique(img)[1] if len(np.unique(img)) > 1 else 1
    binary = img >= thresh

    # Label regions
    labeled_img, num_labels = label(binary)

    return img, labeled_img, num_labels

def analyze_image(image_path):
    """Analyze a single cell mask image and extract morphometric properties"""
    img, labeled_img, num_labels = load_and_get_regions(image_path)

    # Get region properties
    props = regionprops(labeled_img)

    # Extract data for each cell (region)
    data = []
    for i, prop in enumerate(props):
        # Skip small regions (noise)
        if prop.area < 100:
            continue

        # Calculate aspect ratio
        y0, x0 = prop.centroid
        orientation = prop.orientation

        # Convert orientation to degrees
        orientation_deg = np.degrees(orientation)

        # Calculate major and minor axis lengths
        major_axis_length = prop.major_axis_length
        minor_axis_length = prop.minor_axis_length

        # Calculate elongation (aspect ratio)
        aspect_ratio = major_axis_length / minor_axis_length if minor_axis_length > 0 else 0

        # Calculate flow alignment (0 degrees is parallel to flow, 90 degrees is perpendicular)
        # Flow is from left to right, orientation is relative to y-axis
        # Convert to angle relative to x-axis (flow direction)
        flow_alignment = abs(90 - abs(orientation_deg))

        # Add to data
        data.append({
            'Area': prop.area,
            'Perimeter': prop.perimeter,
            'Centroid_X': x0,
            'Centroid_Y': y0,
            'Major_Axis_Length': major_axis_length,
            'Minor_Axis_Length': minor_axis_length,
            'Aspect_Ratio': aspect_ratio,
            'Orientation_Deg': orientation_deg,
            'Flow_Alignment': flow_alignment,
            'Eccentricity': prop.eccentricity,
            'Solidity': prop.solidity,
        })

    return data, labeled_img, img

def visualize_alignment(labeled_img, img, properties, output_path):
    """Visualize the cell orientations relative to flow direction"""
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img, cmap='gray')

    # Draw arrows indicating orientation
    for prop in properties:
        y0, x0 = prop['Centroid_Y'], prop['Centroid_X']
        orientation = np.radians(prop['Orientation_Deg'])

        # Calculate arrow length based on major axis
        length = prop['Major_Axis_Length'] / 2

        # Calculate arrow endpoints
        dx = length * np.sin(orientation)
        dy = -length * np.cos(orientation)

        # Draw arrow
        ax.arrow(x0, y0, dx, dy, head_width=10, head_length=10,
                 fc='red', ec='red', alpha=0.7)

    # Draw flow direction arrow
    ax.arrow(50, 50, 100, 0, head_width=20, head_length=20,
             fc='blue', ec='blue', linewidth=3, alpha=0.7)
    ax.text(50, 30, 'Flow Direction', color='blue', fontsize=12)

    plt.title('Cell Alignment Relative to Flow Direction')
    plt.savefig(output_path)
    plt.close()

def create_alignment_histogram(data_0pa, data_1_4pa, output_path):
    """Create histogram comparing flow alignment between 0Pa and 1.4Pa"""
    alignment_0pa = [cell['Flow_Alignment'] for cell in data_0pa]
    alignment_1_4pa = [cell['Flow_Alignment'] for cell in data_1_4pa]

    fig, ax = plt.subplots(figsize=(10, 6))

    # Create bins from 0 to 90 degrees
    bins = np.linspace(0, 90, 19)  # 5-degree bins

    # Plot histograms
    ax.hist(alignment_0pa, bins=bins, alpha=0.5, label='0 Pa', density=True)
    ax.hist(alignment_1_4pa, bins=bins, alpha=0.5, label='1.4 Pa', density=True)

    ax.set_xlabel('Flow Alignment Angle (degrees)')
    ax.set_ylabel('Frequency')
    ax.set_title('Distribution of Cell Alignment Relative to Flow Direction')

    # Add vertical lines at 0 and 90 degrees with labels
    ax.axvline(x=0, linestyle='--', color='k', alpha=0.3)
    ax.axvline(x=90, linestyle='--', color='k', alpha=0.3)
    ax.text(2, ax.get_ylim()[1]*0.9, 'Parallel to Flow', rotation=90, alpha=0.7)
    ax.text(92, ax.get_ylim()[1]*0.9, 'Perpendicular to Flow', rotation=90, alpha=0.7)

    ax.legend()
    plt.savefig(output_path)
    plt.close()

def create_property_boxplots(all_data_0pa, all_data_1_4pa, output_folder):
    """Create boxplots comparing morphometric properties between 0Pa and 1.4Pa"""
    # Combine all cells from all images
    df_0pa = pd.DataFrame([cell for image_data in all_data_0pa for cell in image_data])
    df_1_4pa = pd.DataFrame([cell for image_data in all_data_1_4pa for cell in image_data])

    # Add pressure column
    df_0pa['Pressure'] = '0 Pa'
    df_1_4pa['Pressure'] = '1.4 Pa'

    # Combine dataframes
    df_combined = pd.concat([df_0pa, df_1_4pa])

    # Properties to plot
    properties = ['Area', 'Perimeter', 'Aspect_Ratio', 'Eccentricity', 'Solidity', 'Flow_Alignment']

    for prop in properties:
        plt.figure(figsize=(8, 6))
        ax = plt.gca()

        df_combined.boxplot(column=prop, by='Pressure', ax=ax)

        plt.title(f'Comparison of {prop.replace("_", " ")}')
        plt.suptitle('')  # Remove default suptitle
        plt.savefig(os.path.join(output_folder, f'boxplot_{prop}.png'))
        plt.close()

    # Save the combined data to CSV
    df_combined.to_csv(os.path.join(output_folder, 'cell_morphometrics_comparison.csv'), index=False)

    # Calculate and save summary statistics
    summary = df_combined.groupby('Pressure').agg(['mean', 'std', 'median', 'min', 'max'])
    summary.to_csv(os.path.join(output_folder, 'cell_morphometrics_summary.csv'))

    return df_combined

def compare_cell_counts(all_data_0pa, all_data_1_4pa, output_path):
    """Compare cell counts between conditions"""
    count_0pa = sum(len(data) for data in all_data_0pa)
    count_1_4pa = sum(len(data) for data in all_data_1_4pa)

    plt.figure(figsize=(6, 8))
    plt.bar(['0 Pa', '1.4 Pa'], [count_0pa, count_1_4pa])
    plt.title('Cell Count Comparison')
    plt.ylabel('Number of Cells')
    plt.savefig(output_path)
    plt.close()

    return count_0pa, count_1_4pa

def main():
    # Get cell mask files organized by pressure
    files_0pa, files_1_4pa = get_cell_masks(input_folder)

    print(f"Found {len(files_0pa)} files for 0Pa and {len(files_1_4pa)} files for 1.4Pa")

    # Analyze all images
    all_data_0pa = []
    all_data_1_4pa = []

    # Process 0Pa images
    for i, file_path in enumerate(files_0pa):
        print(f"Processing 0Pa image {i+1}/{len(files_0pa)}: {os.path.basename(file_path)}")
        data, labeled_img, img = analyze_image(file_path)
        all_data_0pa.append(data)

        # Visualize a few examples
        if i < 3:  # Limit to first 3 images for visualization
            output_path = os.path.join(output_folder, f'alignment_visualization_0Pa_{i+1}.png')
            visualize_alignment(labeled_img, img, data, output_path)

    # Process 1.4Pa images
    for i, file_path in enumerate(files_1_4pa):
        print(f"Processing 1.4Pa image {i+1}/{len(files_1_4pa)}: {os.path.basename(file_path)}")
        data, labeled_img, img = analyze_image(file_path)
        all_data_1_4pa.append(data)

        # Visualize a few examples
        if i < 3:  # Limit to first 3 images for visualization
            output_path = os.path.join(output_folder, f'alignment_visualization_1.4Pa_{i+1}.png')
            visualize_alignment(labeled_img, img, data, output_path)

    # Create alignment histogram
    alignment_histogram_path = os.path.join(output_folder, 'alignment_histogram.png')
    flat_data_0pa = [cell for image_data in all_data_0pa for cell in image_data]
    flat_data_1_4pa = [cell for image_data in all_data_1_4pa for cell in image_data]
    create_alignment_histogram(flat_data_0pa, flat_data_1_4pa, alignment_histogram_path)

    # Create property boxplots
    df_combined = create_property_boxplots(all_data_0pa, all_data_1_4pa, output_folder)

    # Compare cell counts
    cell_count_path = os.path.join(output_folder, 'cell_count_comparison.png')
    count_0pa, count_1_4pa = compare_cell_counts(all_data_0pa, all_data_1_4pa, cell_count_path)

    # Create a summary report
    summary_path = os.path.join(output_folder, 'analysis_summary.txt')
    with open(summary_path, 'w') as f:
        f.write("Cell Morphometric Analysis Summary\n")
        f.write("=================================\n\n")
        f.write(f"Total images analyzed: {len(files_0pa) + len(files_1_4pa)}\n")
        f.write(f"  - 0 Pa images: {len(files_0pa)}\n")
        f.write(f"  - 1.4 Pa images: {len(files_1_4pa)}\n\n")

        f.write(f"Total cells analyzed: {count_0pa + count_1_4pa}\n")
        f.write(f"  - 0 Pa cells: {count_0pa}\n")
        f.write(f"  - 1.4 Pa cells: {count_1_4pa}\n\n")

        # Get alignment stats
        alignment_0pa = [cell['Flow_Alignment'] for cell in flat_data_0pa]
        alignment_1_4pa = [cell['Flow_Alignment'] for cell in flat_data_1_4pa]

        f.write("Flow Alignment Analysis:\n")
        f.write(f"  - 0 Pa mean alignment: {np.mean(alignment_0pa):.2f}° (90° = perpendicular, 0° = parallel)\n")
        f.write(f"  - 1.4 Pa mean alignment: {np.mean(alignment_1_4pa):.2f}° (90° = perpendicular, 0° = parallel)\n\n")

        # Add p-value from statistical test for alignment
        from scipy.stats import ttest_ind
        t_stat, p_value = ttest_ind(alignment_0pa, alignment_1_4pa)
        f.write(f"  - Statistical test for difference in alignment: p-value = {p_value:.5f}\n")
        f.write(f"    (p < 0.05 indicates statistically significant difference)\n\n")

        # Get aspect ratio stats
        aspect_0pa = [cell['Aspect_Ratio'] for cell in flat_data_0pa]
        aspect_1_4pa = [cell['Aspect_Ratio'] for cell in flat_data_1_4pa]

        f.write("Cell Shape Analysis:\n")
        f.write(f"  - 0 Pa mean aspect ratio: {np.mean(aspect_0pa):.2f}\n")
        f.write(f"  - 1.4 Pa mean aspect ratio: {np.mean(aspect_1_4pa):.2f}\n\n")

        # Statistical test for aspect ratio
        t_stat, p_value = ttest_ind(aspect_0pa, aspect_1_4pa)
        f.write(f"  - Statistical test for difference in aspect ratio: p-value = {p_value:.5f}\n")

        f.write("\nInterpretation:\n")
        if p_value < 0.05:
            f.write("The analysis shows a statistically significant difference in cell morphology and alignment between 0 Pa and 1.4 Pa conditions.\n")

            # Interpret alignment
            if np.mean(alignment_1_4pa) > np.mean(alignment_0pa):
                f.write("Cells under 1.4 Pa flow tend to align more perpendicular to the flow direction compared to control cells.\n")
            else:
                f.write("Cells under 1.4 Pa flow tend to align more parallel to the flow direction compared to control cells.\n")

            # Interpret shape
            if np.mean(aspect_1_4pa) > np.mean(aspect_0pa):
                f.write("Cells under 1.4 Pa flow become more elongated compared to control cells.\n")
            else:
                f.write("Cells under 1.4 Pa flow become less elongated compared to control cells.\n")
        else:
            f.write("The analysis does not show a statistically significant difference in cell morphology and alignment between 0 Pa and 1.4 Pa conditions.\n")

    print(f"Analysis complete. Results saved to {output_folder}")

if __name__ == "__main__":
    main()

Found 3 files for 0Pa and 5 files for 1.4Pa
Processing 0Pa image 1/3: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_cell_mask.tif
Processing 0Pa image 2/3: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_cell_mask.tif
Processing 0Pa image 3/3: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq003_cell_mask.tif
Processing 1.4Pa image 1/5: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq001_cell_mask.tif
Processing 1.4Pa image 2/5: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq003_cell_mask.tif
Processing 1.4Pa image 3/5: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq002_cell_mask.tif
Processing 1.4Pa image 4/5: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq004_cell_mask.tif
Processing 1.4Pa image 5/5: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_cell_mask.tif
Analysis complete. Results saved to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3_1.4Pa_18h


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, color
from skimage.measure import regionprops, regionprops_table
import pandas as pd
from scipy.ndimage import label
import cv2
import re

# Path definitions
input_folder = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Cell"
output_folder = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3_1.4Pa_18h"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def extract_pressure_from_filename(filename):
    """Extract pressure value from filename"""
    if "0Pa" in filename:
        return 0.0
    elif "1.4Pa" in filename:
        return 1.4
    else:
        return None

def get_cell_masks(folder_path):
    """Get all cell mask images and organize by pressure"""
    files_0pa = []
    files_1_4pa = []

    print(f"Searching for cell mask files in: {folder_path}")

    try:
        files = os.listdir(folder_path)
        print(f"Found {len(files)} files in directory")

        for filename in files:
            if filename.endswith((".tif", ".tiff")) and "cell_mask" in filename:
                full_path = os.path.join(folder_path, filename)
                print(f"Found mask file: {filename}")
                pressure = extract_pressure_from_filename(filename)

                if pressure == 0.0:
                    files_0pa.append(full_path)
                    print(f"  - Classified as 0Pa")
                elif pressure == 1.4:
                    files_1_4pa.append(full_path)
                    print(f"  - Classified as 1.4Pa")
                else:
                    print(f"  - Could not determine pressure for: {filename}")
    except Exception as e:
        print(f"Error accessing directory: {e}")

    return files_0pa, files_1_4pa

def load_and_get_regions(image_path):
    """Load an image and get the labeled regions"""
    img = io.imread(image_path)

    # If image is RGB, convert to grayscale
    if len(img.shape) > 2 and img.shape[2] == 3:
        img = color.rgb2gray(img)

    # Ensure image is binary
    thresh = np.unique(img)[1] if len(np.unique(img)) > 1 else 1
    binary = img >= thresh

    # Label regions
    labeled_img, num_labels = label(binary)

    return img, labeled_img, num_labels

def analyze_image(image_path):
    """Analyze a single cell mask image and extract morphometric properties"""
    print(f"Analyzing image: {image_path}")
    img, labeled_img, num_labels = load_and_get_regions(image_path)

    # Get region properties
    props = regionprops(labeled_img)

    # Extract data for each cell (region)
    data = []
    for i, prop in enumerate(props):
        # Skip small regions (noise)
        if prop.area < 100:
            continue

        # Calculate aspect ratio
        y0, x0 = prop.centroid
        orientation = prop.orientation

        # Convert orientation to degrees
        orientation_deg = np.degrees(orientation)

        # Calculate major and minor axis lengths
        major_axis_length = prop.major_axis_length
        minor_axis_length = prop.minor_axis_length

        # Calculate elongation (aspect ratio)
        aspect_ratio = major_axis_length / minor_axis_length if minor_axis_length > 0 else 0

        # Calculate flow alignment (0 degrees is parallel to flow, 90 degrees is perpendicular)
        # Flow is from left to right
        # skimage.measure gives orientation as the angle between the x-axis and the major axis
        # Convert to angle relative to flow direction (x-axis)
        flow_alignment = abs(orientation_deg)

        # Normalize to 0-90 degrees range (we don't care if it's +/- direction)
        if flow_alignment > 90:
            flow_alignment = 180 - flow_alignment
        elif flow_alignment < 0:
            flow_alignment = abs(flow_alignment)
            if flow_alignment > 90:
                flow_alignment = 180 - flow_alignment

        # Add to data
        data.append({
            'Area': prop.area,
            'Perimeter': prop.perimeter,
            'Centroid_X': x0,
            'Centroid_Y': y0,
            'Major_Axis_Length': major_axis_length,
            'Minor_Axis_Length': minor_axis_length,
            'Aspect_Ratio': aspect_ratio,
            'Orientation_Deg': orientation_deg,
            'Flow_Alignment': flow_alignment,
            'Eccentricity': prop.eccentricity,
            'Solidity': prop.solidity,
        })

    return data, labeled_img, img

def visualize_alignment(labeled_img, img, properties, output_path):
    """Visualize the cell orientations relative to flow direction"""
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img, cmap='gray')

    # Draw arrows indicating orientation
    for prop in properties:
        y0, x0 = prop['Centroid_Y'], prop['Centroid_X']

        # Get the orientation (already in radians from skimage)
        orientation_rad = np.radians(prop['Orientation_Deg'])

        # Get major axis length
        major_axis = prop['Major_Axis_Length']

        # Calculate endpoints of the major axis
        dx = 0.5 * major_axis * np.cos(orientation_rad)
        dy = 0.5 * major_axis * np.sin(orientation_rad)

        # Draw a line showing the major axis orientation
        ax.plot([x0 - dx, x0 + dx], [y0 - dy, y0 + dy], 'r-', linewidth=2)

        # Draw a small arrow in the center showing direction
        arrow_length = major_axis / 4
        ax.arrow(x0, y0, arrow_length * np.cos(orientation_rad),
                arrow_length * np.sin(orientation_rad),
                head_width=arrow_length/5, head_length=arrow_length/5,
                fc='red', ec='red', alpha=0.7)

        # Add a small text showing the flow alignment angle
        flow_align = prop['Flow_Alignment']
        ax.text(x0 + 5, y0 + 5, f"{flow_align:.1f}°", color='white',
                backgroundcolor='black', fontsize=8)

    # Draw flow direction arrow
    ax.arrow(50, 50, 100, 0, head_width=20, head_length=20,
             fc='blue', ec='blue', linewidth=3, alpha=0.7)
    ax.text(50, 30, 'Flow Direction', color='blue', fontsize=12)

    plt.title('Cell Alignment Relative to Flow Direction')
    plt.savefig(output_path)
    plt.close()

def create_alignment_histogram(data_0pa, data_1_4pa, output_path):
    """Create histogram comparing flow alignment between 0Pa and 1.4Pa"""
    alignment_0pa = [cell['Flow_Alignment'] for cell in data_0pa]
    alignment_1_4pa = [cell['Flow_Alignment'] for cell in data_1_4pa]

    # Create figure with two subplots - histogram and polar rose plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Create bins from 0 to 90 degrees
    bins = np.linspace(0, 90, 19)  # 5-degree bins

    # Plot histograms on first subplot
    ax1.hist(alignment_0pa, bins=bins, alpha=0.5, label='0 Pa', density=True)
    ax1.hist(alignment_1_4pa, bins=bins, alpha=0.5, label='1.4 Pa', density=True)

    ax1.set_xlabel('Flow Alignment Angle (degrees)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Cell Alignment Relative to Flow Direction')

    # Add vertical lines at 0 and 90 degrees with labels
    ax1.axvline(x=0, linestyle='--', color='k', alpha=0.3)
    ax1.axvline(x=90, linestyle='--', color='k', alpha=0.3)
    ax1.text(2, ax1.get_ylim()[1]*0.9, 'Parallel to Flow', rotation=90, alpha=0.7)
    ax1.text(92, ax1.get_ylim()[1]*0.9, 'Perpendicular to Flow', rotation=90, alpha=0.7)

    ax1.legend()

    # Create polar rose plot on second subplot to visualize orientation
    ax2 = plt.subplot(1, 2, 2, polar=True)

    # For the rose plot, we need to duplicate angles to show bidirectional nature
    # and convert to radians
    bins_rad = np.linspace(0, np.pi, 19)  # 10-degree bins in radians

    # Create histogram data
    hist_0pa, _ = np.histogram(np.array(alignment_0pa) * np.pi/180, bins=bins_rad)
    hist_1_4pa, _ = np.histogram(np.array(alignment_1_4pa) * np.pi/180, bins=bins_rad)

    # Normalize histograms
    if len(alignment_0pa) > 0:
        hist_0pa = hist_0pa / len(alignment_0pa)
    if len(alignment_1_4pa) > 0:
        hist_1_4pa = hist_1_4pa / len(alignment_1_4pa)

    # Plot rose diagrams
    width = np.pi/18  # Width of each bin
    bars_0pa = ax2.bar(bins_rad[:-1], hist_0pa, width=width, bottom=0.0, alpha=0.5, label='0 Pa')
    bars_1_4pa = ax2.bar(bins_rad[:-1], hist_1_4pa, width=width, bottom=0.0, alpha=0.5, label='1.4 Pa')

    # Duplicate for bidirectional symmetry (cells don't have direction)
    bars_0pa_mirror = ax2.bar(bins_rad[:-1] + np.pi, hist_0pa, width=width, bottom=0.0, alpha=0.5)
    bars_1_4pa_mirror = ax2.bar(bins_rad[:-1] + np.pi, hist_1_4pa, width=width, bottom=0.0, alpha=0.5)

    # Set the direction labels
    ax2.set_theta_zero_location("E")  # 0 degrees at the right (flow direction)
    ax2.set_theta_direction(-1)  # Clockwise

    # Add directional labels
    ax2.text(0, ax2.get_rmax()*1.1, 'Flow Direction →', ha='center', va='center')
    ax2.text(np.pi/2, ax2.get_rmax()*1.1, '↑ Perpendicular', ha='center', va='center')
    ax2.text(np.pi, ax2.get_rmax()*1.1, '← Flow Direction', ha='center', va='center')
    ax2.text(3*np.pi/2, ax2.get_rmax()*1.1, '↓ Perpendicular', ha='center', va='center')

    ax2.set_title('Rose Diagram of Cell Orientation')
    ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def create_property_boxplots(all_data_0pa, all_data_1_4pa, output_folder):
    """Create boxplots comparing morphometric properties between 0Pa and 1.4Pa"""
    print("Creating comparative boxplots for morphometric properties...")
    # Combine all cells from all images
    df_0pa = pd.DataFrame([cell for image_data in all_data_0pa for cell in image_data])
    df_1_4pa = pd.DataFrame([cell for image_data in all_data_1_4pa for cell in image_data])

    # Add pressure column
    df_0pa['Pressure'] = '0 Pa'
    df_1_4pa['Pressure'] = '1.4 Pa'

    # Combine dataframes
    df_combined = pd.concat([df_0pa, df_1_4pa])

    # Properties to plot
    properties = ['Area', 'Perimeter', 'Aspect_Ratio', 'Eccentricity', 'Solidity', 'Flow_Alignment']

    for prop in properties:
        plt.figure(figsize=(8, 6))
        ax = plt.gca()

        df_combined.boxplot(column=prop, by='Pressure', ax=ax)

        plt.title(f'Comparison of {prop.replace("_", " ")}')
        plt.suptitle('')  # Remove default suptitle
        plt.savefig(os.path.join(output_folder, f'boxplot_{prop}.png'))
        plt.close()

    # Save the combined data to CSV
    df_combined.to_csv(os.path.join(output_folder, 'cell_morphometrics_comparison.csv'), index=False)

    # Calculate and save summary statistics
    summary = df_combined.groupby('Pressure').agg(['mean', 'std', 'median', 'min', 'max'])
    summary.to_csv(os.path.join(output_folder, 'cell_morphometrics_summary.csv'))

    return df_combined

def compare_cell_counts(all_data_0pa, all_data_1_4pa, output_path):
    """Compare cell counts between conditions"""
    count_0pa = sum(len(data) for data in all_data_0pa)
    count_1_4pa = sum(len(data) for data in all_data_1_4pa)

    plt.figure(figsize=(6, 8))
    plt.bar(['0 Pa', '1.4 Pa'], [count_0pa, count_1_4pa])
    plt.title('Cell Count Comparison')
    plt.ylabel('Number of Cells')
    plt.savefig(output_path)
    plt.close()

    return count_0pa, count_1_4pa

def main():
    # Get cell mask files organized by pressure
    files_0pa, files_1_4pa = get_cell_masks(input_folder)

    print(f"Found {len(files_0pa)} files for 0Pa and {len(files_1_4pa)} files for 1.4Pa")

    # Analyze all images
    all_data_0pa = []
    all_data_1_4pa = []

    # Process 0Pa images
    for i, file_path in enumerate(files_0pa):
        print(f"Processing 0Pa image {i+1}/{len(files_0pa)}: {os.path.basename(file_path)}")
        data, labeled_img, img = analyze_image(file_path)
        all_data_0pa.append(data)

        # Visualize a few examples
        if i < 3:  # Limit to first 3 images for visualization
            output_path = os.path.join(output_folder, f'alignment_visualization_0Pa_{i+1}.png')
            visualize_alignment(labeled_img, img, data, output_path)

    # Process 1.4Pa images
    for i, file_path in enumerate(files_1_4pa):
        print(f"Processing 1.4Pa image {i+1}/{len(files_1_4pa)}: {os.path.basename(file_path)}")
        data, labeled_img, img = analyze_image(file_path)
        all_data_1_4pa.append(data)

        # Visualize a few examples
        if i < 3:  # Limit to first 3 images for visualization
            output_path = os.path.join(output_folder, f'alignment_visualization_1.4Pa_{i+1}.png')
            visualize_alignment(labeled_img, img, data, output_path)

    # Create alignment histogram
    alignment_histogram_path = os.path.join(output_folder, 'alignment_histogram.png')
    flat_data_0pa = [cell for image_data in all_data_0pa for cell in image_data]
    flat_data_1_4pa = [cell for image_data in all_data_1_4pa for cell in image_data]
    create_alignment_histogram(flat_data_0pa, flat_data_1_4pa, alignment_histogram_path)

    # Create property boxplots
    df_combined = create_property_boxplots(all_data_0pa, all_data_1_4pa, output_folder)

    # Compare cell counts
    cell_count_path = os.path.join(output_folder, 'cell_count_comparison.png')
    count_0pa, count_1_4pa = compare_cell_counts(all_data_0pa, all_data_1_4pa, cell_count_path)

    # Create a summary report
    summary_path = os.path.join(output_folder, 'analysis_summary.txt')
    with open(summary_path, 'w') as f:
        f.write("Cell Morphometric Analysis Summary\n")
        f.write("=================================\n\n")
        f.write(f"Total images analyzed: {len(files_0pa) + len(files_1_4pa)}\n")
        f.write(f"  - 0 Pa images: {len(files_0pa)}\n")
        f.write(f"  - 1.4 Pa images: {len(files_1_4pa)}\n\n")

        f.write(f"Total cells analyzed: {count_0pa + count_1_4pa}\n")
        f.write(f"  - 0 Pa cells: {count_0pa}\n")
        f.write(f"  - 1.4 Pa cells: {count_1_4pa}\n\n")

        # Get alignment stats
        alignment_0pa = [cell['Flow_Alignment'] for cell in flat_data_0pa]
        alignment_1_4pa = [cell['Flow_Alignment'] for cell in flat_data_1_4pa]

        f.write("Flow Alignment Analysis:\n")
        f.write(f"  - 0 Pa mean alignment: {np.mean(alignment_0pa):.2f}° (90° = perpendicular, 0° = parallel)\n")
        f.write(f"  - 1.4 Pa mean alignment: {np.mean(alignment_1_4pa):.2f}° (90° = perpendicular, 0° = parallel)\n\n")

        # Add p-value from statistical test for alignment
        from scipy.stats import ttest_ind
        t_stat, p_value = ttest_ind(alignment_0pa, alignment_1_4pa)
        f.write(f"  - Statistical test for difference in alignment: p-value = {p_value:.5f}\n")
        f.write(f"    (p < 0.05 indicates statistically significant difference)\n\n")

        # Get aspect ratio stats
        aspect_0pa = [cell['Aspect_Ratio'] for cell in flat_data_0pa]
        aspect_1_4pa = [cell['Aspect_Ratio'] for cell in flat_data_1_4pa]

        f.write("Cell Shape Analysis:\n")
        f.write(f"  - 0 Pa mean aspect ratio: {np.mean(aspect_0pa):.2f}\n")
        f.write(f"  - 1.4 Pa mean aspect ratio: {np.mean(aspect_1_4pa):.2f}\n\n")

        # Statistical test for aspect ratio
        t_stat, p_value = ttest_ind(aspect_0pa, aspect_1_4pa)
        f.write(f"  - Statistical test for difference in aspect ratio: p-value = {p_value:.5f}\n")

        f.write("\nInterpretation:\n")
        if p_value < 0.05:
            f.write("The analysis shows a statistically significant difference in cell morphology and alignment between 0 Pa and 1.4 Pa conditions.\n")

            # Interpret alignment
            if np.mean(alignment_1_4pa) > np.mean(alignment_0pa):
                f.write("Cells under 1.4 Pa flow tend to align more perpendicular to the flow direction compared to control cells.\n")
            else:
                f.write("Cells under 1.4 Pa flow tend to align more parallel to the flow direction compared to control cells.\n")

            # Interpret shape
            if np.mean(aspect_1_4pa) > np.mean(aspect_0pa):
                f.write("Cells under 1.4 Pa flow become more elongated compared to control cells.\n")
            else:
                f.write("Cells under 1.4 Pa flow become less elongated compared to control cells.\n")
        else:
            f.write("The analysis does not show a statistically significant difference in cell morphology and alignment between 0 Pa and 1.4 Pa conditions.\n")

    print(f"Analysis complete. Results saved to {output_folder}")

if __name__ == "__main__":
    main()